In [ ]:
import datacube
import xarray as xr
import dask
from dask.distributed import performance_report
from dask.distributed import Client

from datacube.utils import masking
## https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py

## Connect to ODC
dc = datacube.Datacube()

## Start local Dask cluster
## https://docs.dask.org/en/latest/setup/single-distributed.html#localcluster
n_workers = 1
n_threads = 24
client = Client(processes=False, 
                n_workers=n_workers, 
                threads_per_worker=n_threads)

In [ ]:
from utils.geometry import geojson_x_y
aoi = geojson_x_y("./misc/subset.geojson")
x = aoi[0]
y = aoi[1]

In [ ]:
chunk_size = 1000

ds = dc.load(product=f"s1_ARD_asc",
             x=x,
             y=y,
             dask_chunks={'time':-1, 'y':chunk_size, 'x':chunk_size})

ds.VH

In [ ]:
nodata = ds.VH.attrs['nodata']
count = xr.where(ds.VH > nodata, True, False).sum(dim="time")
count

In [ ]:
%%time
with performance_report(filename=f"./reports/dask_test_obs_perpixel_SAR_asc__{n_workers}-{n_threads}_{chunk_size}-{chunk_size}.html"):
    count = count.compute()